<a href="https://colab.research.google.com/github/lucasuff/Vies-de-genero-em-PLN-em-portugues/blob/main/bertFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install transformers

In [ ]:
from transformers import BertTokenizer, BertForPreTraining
import torch
import re

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model = BertForPreTraining.from_pretrained('neuralmind/bert-base-portuguese-cased').to(device)
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

Some weights of BertForPreTraining were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def clean_text(contents):
  contents = "".join(re.findall(r"[0-9a-zA-Z\u00C0-\u00FF,\s\?\!\.\n]+", contents))
  contents = contents.replace("\t", "").replace("\xa0", "")
  contents =  re.sub("\s{2,}", " ", contents)
  #contents =  re.sub(r"([.?!])\s*", r"\1\n", contents)
  return contents

In [ ]:
import json

file = open("/content/drive/MyDrive/curso_word2vec/contents_woman.json", "r")
trip_contents = "\n".join(json.load(file))
trip_contents = clean_text(trip_contents)
file.close()

In [ ]:
file = open("/content/drive/MyDrive/curso_word2vec/elpais_woman_contents.json", "r")
elpais_contents = " ".join(json.load(file))
elpais_contents = clean_text(elpais_contents)
file.close()

In [ ]:
data = trip_contents + " " + elpais_contents
text = data.split('\n')

In [ ]:
bag = [sentence for para in text for sentence in para.split(".") if sentence != ""]
bag_size = len(bag)

In [ ]:
import random

sentence_a = []
sentence_b = []
label = []

for paragraph in text:
  sentences = [
               sentence for sentence in paragraph.split(".") if sentence != ""
  ]
  num_sentences = len(sentences)
  if num_sentences > 1:
    start = random.randint(0, num_sentences-2)
    sentence_a.append(sentences[start])
    if random.random() > 0.5:
      sentence_b.append(sentences[start+1])
      label.append(0)
    else:
      sentence_b.append(bag[random.randint(0, bag_size-1)])
      label.append(1)


In [ ]:
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
inputs['next_sentence_label'] = torch.LongTensor([label]).T

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'next_sentence_label', 'labels'])

In [ ]:
rand = torch.rand(inputs.input_ids.shape)

In [ ]:
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
for i in range(inputs.input_ids.shape[0]):
  selection = torch.flatten(mask_arr[i].nonzero()).tolist()
  inputs.input_ids[i, selection] = 103

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
device

device(type='cuda')

In [ ]:
model.train()

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [ ]:
from transformers import AdamW

optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
from tqdm import tqdm

for epoch in range(2):
  loop = tqdm(loader, leave=True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    next_sentence_label = batch['next_sentence_label'].to(device)
    labels = batch['labels'].to(device)
    outputs = model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, next_sentence_label=next_sentence_label, labels=labels)
    loss = outputs.loss
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch}')
    loop.set_postfix(los=loss.item())


  0%|          | 0/903 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 1: 100%|██████████| 903/903 [28:16<00:00,  1.88s/it, los=0.118]


In [ ]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'next_sentence_label', 'labels'])

In [ ]:
model.eval()

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [ ]:
model.save_pretrained("/content/drive/MyDrive/curso_word2vec/bertFineTunedWomanModel")